<a href="https://colab.research.google.com/github/PythonDecorator/Amazon-EC2-with-github-actions/blob/main/Tennis_Analysis_System_with_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install pakages

In [3]:
!pip install -q ultralytics
!pip install -q lap

# Imports

In [20]:
import os
import pickle
import numpy as np

# traking and object detection
from ultralytics import YOLO
import lap
import cv2

# courtline detection
import torch
import torchvision.transforms as transforms
from torchvision import models

# Display Video and other results
from IPython.display import HTML
from base64 import b64encode


In [5]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
INPUT_VIDEO_PATH = "/content/drive/MyDrive/MY WORK/ML/07-Tennis-Analysis-System/input_videos/input_video.mp4"
YOLO_MODEL = "yolo12n.pt"


# Perform Object Detection/Tracking - using YOLO

In [8]:
#Load the YOLOv12 Model
model = YOLO(YOLO_MODEL)

#Predictions
results = model.track(source=INPUT_VIDEO_PATH, save=True, persist=True, stream=True)

In [9]:
for frame_idx, r in enumerate(results):
    frame = r.orig_img  # numpy array (H, W, 3) BGR
    boxes = r.boxes

    if boxes is None or boxes.xyxy is None or len(boxes) == 0:
        continue

    xyxy = boxes.xyxy.cpu().numpy()  # (N, 4)
    ids = None
    if boxes.id is not None:
        ids = boxes.id.cpu().numpy().astype(int)  # (N,)


video 1/1 (frame 1/214) /content/drive/MyDrive/MY WORK/ML/07-Tennis-Analysis-System/input_videos/input_video.mp4: 384x640 5 persons, 366.8ms
video 1/1 (frame 2/214) /content/drive/MyDrive/MY WORK/ML/07-Tennis-Analysis-System/input_videos/input_video.mp4: 384x640 5 persons, 147.8ms
video 1/1 (frame 3/214) /content/drive/MyDrive/MY WORK/ML/07-Tennis-Analysis-System/input_videos/input_video.mp4: 384x640 5 persons, 142.7ms
video 1/1 (frame 4/214) /content/drive/MyDrive/MY WORK/ML/07-Tennis-Analysis-System/input_videos/input_video.mp4: 384x640 5 persons, 136.3ms
video 1/1 (frame 5/214) /content/drive/MyDrive/MY WORK/ML/07-Tennis-Analysis-System/input_videos/input_video.mp4: 384x640 5 persons, 150.8ms
video 1/1 (frame 6/214) /content/drive/MyDrive/MY WORK/ML/07-Tennis-Analysis-System/input_videos/input_video.mp4: 384x640 5 persons, 138.4ms
video 1/1 (frame 7/214) /content/drive/MyDrive/MY WORK/ML/07-Tennis-Analysis-System/input_videos/input_video.mp4: 384x640 5 persons, 137.8ms
video 1/1 (f

> ## Display the output video

In [13]:
# Input & output paths
save_path = "/content/runs/detect/track/input_video.avi"
compressed_path = "/content/result_compressed.mp4"

# Convert + compress (H.264, web-safe)
os.system(
    f"ffmpeg -y -i {save_path} "
    f"-vcodec libx264 -pix_fmt yuv420p -movflags +faststart "
    f"-crf 23 {compressed_path}"
)

0

In [15]:
def display_video(path, width=900, height="auto"):
    video_bytes = open(path, "rb").read()
    encoded = b64encode(video_bytes).decode()

    html = f"""
    <video width="{width}" height="{height}" controls autoplay loop>
        <source src="data:video/mp4;base64,{encoded}" type="video/mp4">
    </video>
    """
    return HTML(html)

display_video("/content/result_compressed.mp4", width=900)


# Tenis Ball Detector

In [26]:
!pip install -q roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 90.7 MB/s eta 0:00:00


In [27]:
import shutil
from roboflow import Roboflow

In [ ]:
rf = Roboflow(api_key="8joVH7TL2k5V6uScFBEy")
project = rf.workspace("viren-dhanwani").project("tennis-ball-detection")
version = project.version(6)
dataset = version.download("yolov5")

In [ ]:
shutil.move("tennis-ball-detection-6/train",
"tennis-ball-detection-6/tennis-ball-detection-6/train",
)
shutil.move("tennis-ball-detection-6/test",
"tennis-ball-detection-6/tennis-ball-detection-6/test",
)
shutil.move("tennis-ball-detection-6/valid",
"tennis-ball-detection-6/tennis-ball-detection-6/valid",
)


In [ ]:
!yolo task=detect mode=train model=yolov5l6u.pt data={dataset.location}/data.yaml epochs=100 imgsz=640

# KeyPoints Detector

In [28]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import cv2
import numpy as np

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

> download and unzip dataset

In [ ]:
!gdown "https://drive.google.com/uc?id=1O4-mRTgHK4AntMNDdQmOP-jnxXs9agnQ&confirm=t"

In [ ]:
!unzip "tennis_court_det_dataset.zip"

In [ ]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [ ]:
train_dataset = KeypointsDataset("data/images", "data/data_train.json")
val_dataset = KeypointsDataset("data/images", "data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size = 8, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 8, shuffle = True)

> Create Model

In [ ]:
model = models.resnet50(pretrained = True)
model.fc = torch.nn.Linear(model.fc.in_features, 14 * 2)

> Move model to device

In [ ]:
model = model.to(device)

> Train Model

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

In [ ]:
epochs=50
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

In [ ]:
torch.save(model.state_dict(), "keypoints_model_50.pth")

# Utils

In [21]:
def get_center_bbox(bbox):
    x1, y1, x2, y2 = bbox
    center_x = int((x1 + x2)/2)
    center_y = int((y1 + y2)/2)
    return (center_x, center_y)

def measure_distance(p1, p2):
    return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)**0.5

def read_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break
    cap.release()
    return frames

def save_video(output_video_frames, output_video_path):
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out=cv2.VideoWriter(output_video_path, fourcc, 24, (output_video_frames[0].shape[1], output_video_frames[0].shape[0]))
    for frame in output_video_frames:
        out.write(frame)
    out.release()


# CourtLine Detector

In [22]:
class CourtLineDetector:
    def __init__(self, model_path):
        self.model = models.resnet50(pretrained = True)
        self.model.fc = torch.nn.Linear(self.model.fc.in_features, 14*2)
        self.model.load_state_dict(torch.load(model_path, map_location = "cpu"))
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
        ])

    def predict(self, image):
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_tensor = self.transform(image_rgb).unsqueeze(0)
        with torch.no_grad():
            outputs = self.model(image_tensor)
        keypoints = outputs.squeeze().cpu().numpy()
        original_height, original_width = image.shape[:2]
        keypoints[::2] *= original_width / 224.0
        keypoints[1::2] *= original_height / 224.0
        return keypoints

    #Plot Keypoints on the image
    def draw_keypoints(self, image, keypoints):
        for i in range(0, len(keypoints), 2):
            x = int(keypoints[i])
            y = int(keypoints[i + 1])
            cv2.putText(image, str(i//2), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
            cv2.circle(image, (x,y), 5, (0,0,255), -1)
        return image

    #Plot/ Draw keypoints on the video
    def draw_keypoints_on_video(self, video_frames, keypoints):
        output_video_frames = []
        for frame in video_frames:
            frame = self.draw_keypoints(frame, keypoints)
            output_video_frames.append(frame)
        return output_video_frames

# Player Tracker

In [23]:
class PlayerTracker:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
    def choose_and_filter_players(self, court_keypoints, player_detections):
        player_detections_first_frame = player_detections[0]
        chosen_player = self.choose_players(court_keypoints, player_detections_first_frame)
        filtered_player_detections = []
        for player_det in player_detections:
            filtered_player_dict = {track_id : bbox for track_id, bbox in player_det.items() if track_id in chosen_player}
            filtered_player_detections.append(filtered_player_dict)
        return filtered_player_detections

    def choose_players(self, court_keypoints, player_dict):
        distances = []
        for track_id, bbox in player_dict.items():
            player_center = get_center_bbox(bbox)
            min_distance = float('inf')
            for i in range(0, len(court_keypoints), 2):
                court_keypoint = (court_keypoints[i], court_keypoints[i + 1])
                distance = measure_distance(player_center, court_keypoint)
                if distance < min_distance:
                    min_distance = distance
            distances.append((track_id, min_distance))
        #Sort the distances in ascending order
        distances.sort(key = lambda x: x[1])
        #Choose the first two tracks
        chosen_players = [distances[0][0], distances[1][0]]
        return chosen_players

    def detect_frame(self, frame):
        results = self.model.track(frame, persist=True)[0]
        class_names = results.names
        player_dict = {}
        for box in results.boxes:
            track_id = int(box.id.tolist()[0])
            result = box.xyxy.tolist()[0]
            class_ids = box.cls.tolist()[0]
            det_class_names = class_names[class_ids]
            if det_class_names == "person":
                player_dict[track_id] = result
        return player_dict
    def detect_frames(self, frames, read_from_stub=False, stub_path=None):
        player_detections = []
        if read_from_stub and stub_path is not None:
            with open(stub_path, 'rb') as f:
                player_detections = pickle.load(f)
                return player_detections
        for frame in frames:
            player_dict = self.detect_frame(frame)
            player_detections.append(player_dict)
        if stub_path is not None:
            with open(stub_path, 'wb') as f:
                pickle.dump(player_detections, f)
        return player_detections
    def draw_bboxes(self, video_frames, player_detections):
        output_video_frames = []
        for frame, player_dict in zip(video_frames, player_detections):
            for track_id, bbox in player_dict.items():
                x1, y1, x2, y2 = bbox
                cv2.putText(frame, f"Player ID: {track_id}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)
            output_video_frames.append(frame)
        return output_video_frames

# Ball Tracker

In [24]:
class BallTracker:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
    def detect_frame(self, frame):
        results = self.model.predict(frame, conf=0.15)[0]
        ball_dict = {}
        for box in results.boxes:
            result = box.xyxy.tolist()[0]
            ball_dict[1] = result
        return ball_dict
    def detect_frames(self, frames, read_from_stub=False, stub_path=None):
        ball_detections = []
        if read_from_stub and stub_path is not None:
            with open(stub_path, 'rb') as f:
                ball_detections = pickle.load(f)
                return ball_detections
        for frame in frames:
            ball_dict = self.detect_frame(frame)
            ball_detections.append(ball_dict)
        if stub_path is not None:
            with open(stub_path, 'wb') as f:
                pickle.dump(ball_detections, f)
        return ball_detections
    def draw_bboxes(self, video_frames, ball_detections):
        output_video_frames = []
        for frame, ball_dict in zip(video_frames, ball_detections):
            for track_id, bbox in ball_dict.items():
                x1, y1, x2, y2 = bbox
                cv2.putText(frame, f"Ball ID: {track_id}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,255), 2)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,255), 2)
            output_video_frames.append(frame)
        return output_video_frames

# Pipeline

In [ ]:
def main():
    #Read the Input Video
    input_video_path = "input_videos/input_video.mp4"
    video_frames = read_video(input_video_path)

    #Detect and Track Players
    player_tracker = PlayerTracker(model_path = "yolo12n.pt")
    player_detections = player_tracker.detect_frames(video_frames, read_from_stub=True, stub_path="tracker_stubs/player_detection.pkl")

    #Detect Tennis Ball
    ball_tracker = BallTracker(model_path = "models/tennis_ball_best.pt")
    ball_detections = ball_tracker.detect_frames(video_frames, read_from_stub=True, stub_path="tracker_stubs/ball_detection.pkl")

    #Detect Tennis Court Keypoints
    court_model_path = "models/keypoints_model_50.pth"
    court_line_detector = CourtLineDetector(court_model_path)
    court_keypoints = court_line_detector.predict(video_frames[0])

    #Choose & Filter Players
    player_detections = player_tracker.choose_and_filter_players(court_keypoints, player_detections)

    #Draw Output
    #Draw Player Bounding Boxes
    output_video_frames = player_tracker.draw_bboxes(video_frames, player_detections)
    #Draw Tennis Ball Bounding Boxes
    output_video_frames = ball_tracker.draw_bboxes(output_video_frames, ball_detections)
    #Draw Tennis Court Keypoints
    output_video_frames = court_line_detector.draw_keypoints_on_video(output_video_frames, court_keypoints)

    #Save Video
    save_video(output_video_frames, "output_videos/output.avi")